# Tabular Baselines - Non-GNN Models

**Goal:** Train traditional ML models on node features only (no graph structure)

**Models:**
1. Logistic Regression (Linear baseline)
2. Random Forest (Tree ensemble)
3. XGBoost (Gradient boosting)
4. MLP (Neural network without graph)

**Data:** Same 182 features, same temporal splits, **NO graph edges**

In [ ]:
import torch
import numpy as np
import pandas as pd
import json
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    average_precision_score,
    roc_auc_score,
    f1_score,
    precision_recall_curve,
    classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ All imports successful!")

## 1. Load Data (Features Only - No Graph!)

In [ ]:
# Load processed data from GNN pipeline
data_path = Path('/kaggle/input/elliptic-fraud-detection')

# Load the PyG data object that was created
import sys
sys.path.append('/kaggle/working')

# Actually, let's just load from the splits we already have
# We'll use the same data loader but extract only features

print("Loading Elliptic++ dataset...")

# Load raw CSVs
features_df = pd.read_csv(data_path / 'txs_features.csv')
classes_df = pd.read_csv(data_path / 'txs_classes.csv')

print(f"Features shape: {features_df.shape}")
print(f"Classes shape: {classes_df.shape}")

# Merge
df = features_df.merge(classes_df, on='txId')
print(f"Merged shape: {df.shape}")

# Normalize timestamp column to 'timestamp'
ts_candidates = ['Time step','time_step','timestamp','time','timestep']
for c in ts_candidates:
    if c in df.columns:
        if c != 'timestamp':
            df = df.rename(columns={c: 'timestamp'})
        break
else:
    raise KeyError(f"No timestamp column found in merged DataFrame; columns: {list(df.columns)[:25]}")

# Filter labeled only
df_labeled = df[df['class'].isin(['1', '2'])].copy()
df_labeled['label'] = (df_labeled['class'] == '2').astype(int)  # 2=fraud, 1=legit

print(f"Labeled transactions: {len(df_labeled)}")
print(f"Fraud percentage: {df_labeled['label'].mean()*100:.2f}%")

## 2. Create Temporal Splits (Same as GNN Models)

In [ ]:
# Sort by timestamp
df_labeled = df_labeled.sort_values('timestamp')

# Split: 60% train, 20% val, 20% test (temporal)
n = len(df_labeled)
train_size = int(0.6 * n)
val_size = int(0.2 * n)

train_df = df_labeled.iloc[:train_size]
val_df = df_labeled.iloc[train_size:train_size+val_size]
test_df = df_labeled.iloc[train_size+val_size:]

print(f"\nTemporal Splits:")
print(f"Train: {len(train_df)} ({len(train_df)/n*100:.1f}%) | Fraud: {train_df['label'].mean()*100:.2f}%")
print(f"Val:   {len(val_df)} ({len(val_df)/n*100:.1f}%) | Fraud: {val_df['label'].mean()*100:.2f}%")
print(f"Test:  {len(test_df)} ({len(test_df)/n*100:.1f}%) | Fraud: {test_df['label'].mean()*100:.2f}%")

In [ ]:
# Prepare features and labels
feature_cols = [col for col in df_labeled.columns if col.startswith('LF') or col.startswith('AF')]

X_train = train_df[feature_cols].values
y_train = train_df['label'].values

X_val = val_df[feature_cols].values
y_val = val_df['label'].values

X_test = test_df[feature_cols].values
y_test = test_df['label'].values

print(f"\nFeature matrix shapes:")
print(f"X_train: {X_train.shape}")
print(f"X_val:   {X_val.shape}")
print(f"X_test:  {X_test.shape}")
print(f"\nNumber of features: {len(feature_cols)}")

## 3. Feature Sanitization (Same as GNN Models)

In [ ]:
# Handle NaN and Inf (same preprocessing as GNN models)
print("Sanitizing features...")

# Replace inf with NaN
X_train = np.where(np.isinf(X_train), np.nan, X_train)
X_val = np.where(np.isinf(X_val), np.nan, X_val)
X_test = np.where(np.isinf(X_test), np.nan, X_test)

# Fill NaN with 0
X_train = np.nan_to_num(X_train, nan=0.0)
X_val = np.nan_to_num(X_val, nan=0.0)
X_test = np.nan_to_num(X_test, nan=0.0)

# Normalize (fit on train only)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Final NaN check
X_train = np.nan_to_num(X_train, nan=0.0)
X_val = np.nan_to_num(X_val, nan=0.0)
X_test = np.nan_to_num(X_test, nan=0.0)

print("✅ Features sanitized and normalized")
print(f"Train NaN count: {np.isnan(X_train).sum()}")
print(f"Train Inf count: {np.isinf(X_train).sum()}")

## 4. Evaluation Function

In [ ]:
def evaluate_model(y_true, y_probs, model_name):
    """
    Evaluate model using same metrics as GNN models
    """
    # PR-AUC (primary metric for imbalanced data)
    pr_auc = average_precision_score(y_true, y_probs)
    
    # ROC-AUC
    roc_auc = roc_auc_score(y_true, y_probs)
    
    # Find optimal threshold using F1 score
    precision, recall, thresholds = precision_recall_curve(y_true, y_probs)
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
    best_idx = np.argmax(f1_scores)
    best_threshold = thresholds[best_idx] if best_idx < len(thresholds) else 0.5
    best_f1 = f1_scores[best_idx]
    
    # Recall at top-K%
    def recall_at_k(y_true, y_probs, k_percent):
        k = int(len(y_true) * k_percent / 100)
        top_k_idx = np.argsort(y_probs)[-k:]
        return y_true[top_k_idx].sum() / y_true.sum()
    
    recall_0_5 = recall_at_k(y_true, y_probs, 0.5)
    recall_1_0 = recall_at_k(y_true, y_probs, 1.0)
    recall_2_0 = recall_at_k(y_true, y_probs, 2.0)
    
    metrics = {
        'pr_auc': float(pr_auc),
        'roc_auc': float(roc_auc),
        'f1': float(best_f1),
        'threshold': float(best_threshold),
        'recall@0.5%': float(recall_0_5),
        'recall@1.0%': float(recall_1_0),
        'recall@2.0%': float(recall_2_0)
    }
    
    print(f"\n{'='*60}")
    print(f"{model_name} - Test Set Results")
    print(f"{'='*60}")
    print(f"PR-AUC:       {pr_auc:.4f}")
    print(f"ROC-AUC:      {roc_auc:.4f}")
    print(f"F1 Score:     {best_f1:.4f} (threshold: {best_threshold:.4f})")
    print(f"Recall@0.5%:  {recall_0_5:.4f}")
    print(f"Recall@1.0%:  {recall_1_0:.4f}")
    print(f"Recall@2.0%:  {recall_2_0:.4f}")
    print(f"{'='*60}\n")
    
    return metrics

## 5. Model 1: Logistic Regression

In [ ]:
print("\n" + "="*60)
print("Training Logistic Regression...")
print("="*60)

# Calculate class weights (to handle imbalance)
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array([0, 1]),
    y=y_train
)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

print(f"Class weights: {class_weight_dict}")

# Train
lr = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=42,
    n_jobs=-1
)

lr.fit(X_train, y_train)
print("✅ Training complete!")

# Predict
y_probs_lr = lr.predict_proba(X_test)[:, 1]

# Evaluate
lr_metrics = evaluate_model(y_test, y_probs_lr, "Logistic Regression")

## 6. Model 2: Random Forest

In [ ]:
print("\n" + "="*60)
print("Training Random Forest...")
print("="*60)

rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=20,
    min_samples_leaf=10,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

rf.fit(X_train, y_train)
print("✅ Training complete!")

# Predict
y_probs_rf = rf.predict_proba(X_test)[:, 1]

# Evaluate
rf_metrics = evaluate_model(y_test, y_probs_rf, "Random Forest")

## 7. Model 3: XGBoost (Expected Best Performer)

In [ ]:
print("\n" + "="*60)
print("Training XGBoost...")
print("="*60)

# Calculate scale_pos_weight
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"scale_pos_weight: {scale_pos_weight:.2f}")

xgb = XGBClassifier(
    n_estimators=200,
    max_depth=8,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    tree_method='hist',  # Fast
    eval_metric='aucpr',
    early_stopping_rounds=20,
    n_jobs=-1
)

# Train with validation set for early stopping
xgb.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=10
)

print(f"\n✅ Training complete! Best iteration: {xgb.best_iteration}")

# Predict
y_probs_xgb = xgb.predict_proba(X_test)[:, 1]

# Evaluate
xgb_metrics = evaluate_model(y_test, y_probs_xgb, "XGBoost")

## 8. Model 4: MLP (Neural Network)

In [ ]:
print("\n" + "="*60)
print("Training MLP (Multi-Layer Perceptron)...")
print("="*60)

mlp = MLPClassifier(
    hidden_layer_sizes=(256, 128, 64),
    activation='relu',
    solver='adam',
    alpha=0.001,  # L2 regularization
    batch_size=512,
    learning_rate_init=0.001,
    max_iter=100,
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=10,
    random_state=42,
    verbose=True
)

mlp.fit(X_train, y_train)
print(f"\n✅ Training complete! Iterations: {mlp.n_iter_}")

# Predict
y_probs_mlp = mlp.predict_proba(X_test)[:, 1]

# Evaluate
mlp_metrics = evaluate_model(y_test, y_probs_mlp, "MLP")

## 9. Comparison with GNN Models

In [ ]:
# GNN results from M3/M4
gnn_results = {
    'GCN': {
        'pr_auc': 0.1976,
        'roc_auc': 0.7627,
        'f1': 0.2487,
        'recall@1.0%': 0.0613
    },
    'GraphSAGE': {
        'pr_auc': 0.4483,
        'roc_auc': 0.8210,
        'f1': 0.4527,
        'recall@1.0%': 0.1478
    },
    'GAT': {
        'pr_auc': 0.1839,
        'roc_auc': 0.7942,
        'f1': 0.2901,
        'recall@1.0%': 0.0126
    }
}

# Create comparison DataFrame
comparison_data = []

# Add GNN models
for model_name, metrics in gnn_results.items():
    comparison_data.append({
        'Model': model_name,
        'Type': 'GNN',
        'PR-AUC': metrics['pr_auc'],
        'ROC-AUC': metrics['roc_auc'],
        'F1': metrics['f1'],
        'Recall@1%': metrics['recall@1.0%']
    })

# Add ML models
ml_results = {
    'Logistic Regression': lr_metrics,
    'Random Forest': rf_metrics,
    'XGBoost': xgb_metrics,
    'MLP': mlp_metrics
}

for model_name, metrics in ml_results.items():
    comparison_data.append({
        'Model': model_name,
        'Type': 'ML',
        'PR-AUC': metrics['pr_auc'],
        'ROC-AUC': metrics['roc_auc'],
        'F1': metrics['f1'],
        'Recall@1%': metrics['recall@1.0%']
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('PR-AUC', ascending=False)

print("\n" + "="*80)
print("FINAL COMPARISON: ALL MODELS")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

# Find best model
best_model = comparison_df.iloc[0]
print(f"\n🏆 BEST MODEL: {best_model['Model']} ({best_model['Type']})")
print(f"   PR-AUC: {best_model['PR-AUC']:.4f}")
print(f"   ROC-AUC: {best_model['ROC-AUC']:.4f}")
print(f"   F1: {best_model['F1']:.4f}")
print(f"   Recall@1%: {best_model['Recall@1%']:.4f}")

## 10. Visualization

In [ ]:
# Bar chart comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics_to_plot = ['PR-AUC', 'ROC-AUC', 'F1', 'Recall@1%']
colors = ['#2ecc71' if t == 'GNN' else '#3498db' for t in comparison_df['Type']]

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx // 2, idx % 2]
    
    bars = ax.barh(comparison_df['Model'], comparison_df[metric], color=colors)
    ax.set_xlabel(metric, fontsize=12, fontweight='bold')
    ax.set_title(f'{metric} Comparison', fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # Add value labels
    for i, (bar, val) in enumerate(zip(bars, comparison_df[metric])):
        ax.text(val + 0.01, i, f'{val:.4f}', va='center', fontsize=10)

# Legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#2ecc71', label='GNN (uses graph)'),
    Patch(facecolor='#3498db', label='ML (features only)')
]
fig.legend(handles=legend_elements, loc='upper center', ncol=2, fontsize=12)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig('all_models_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualization saved: all_models_comparison.png")

## 11. Save Results

In [ ]:
# Save individual model metrics
for model_name, metrics in ml_results.items():
    filename = f"{model_name.lower().replace(' ', '_')}_metrics.json"
    with open(filename, 'w') as f:
        json.dump(metrics, f, indent=2)
    print(f"✅ Saved: {filename}")

# Save comparison table
comparison_df.to_csv('all_models_comparison.csv', index=False)
print("\n✅ Saved: all_models_comparison.csv")

print("\n" + "="*80)
print("ALL MODELS TRAINED AND EVALUATED SUCCESSFULLY!")
print("="*80)

## 12. Key Insights

**Questions to Answer:**

1. **Does the graph help?**
   - If best GNN >> best ML → Graph is essential
   - If best ML ≈ best GNN → Features contain graph info already
   - If best ML > best GNN → Graph adds noise

2. **Which model to deploy?**
   - Best PR-AUC (primary metric for fraud)
   - Best Recall@1% (catch most fraud in top predictions)
   - Balance performance vs complexity

3. **What did we learn?**
   - Feature importance (from RF/XGBoost)
   - Graph structure value (GNN vs ML gap)
   - Temporal distribution shift handling